# Assignment 5 - Issue #854

Fernando Mendoza | Adrea Pezo | Michel Cotrina | Estefanny Gil | Armando Ore

The script should not give an error. Any mistake will be consider as 0.
<br>We want an excel file with all the available presidential elections in this webpage.
https://infogob.jne.gob.pe/Eleccion
<br>The final excel should have these 3 columns 
(Elecciones | ORGANIZACIÓN POLÍTICA | TOTAL VOTOS)

In [26]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install openpyxl

# Run the code line depending on the internet browser you are using (Chrome or Firefox)

## a. Setup for Chrome only

In [5]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd
from openpyxl import load_workbook

# Path for google Chrome
chrome_driver_path = r'chromedriver-win64/chromedriver.exe'
# Configue the service for google chrome
service = Service(executable_path=chrome_driver_path)
# Start the WebDriver for google chrome
driver = webdriver.Chrome(service=service)

## b. Setup for FireFox only

In [36]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd
from openpyxl import load_workbook

# Path for geckodriver (Firefox)
gecko_driver_path = r'geckodriver-v0.34.0-win64\geckodriver.exe'

# Configure the service for Firefox
service = FirefoxService(executable_path=gecko_driver_path)

# Start the WebDriver for Firefox
driver = webdriver.Firefox(service=service)


# 3. Iteration

In [7]:
# Maximize window
driver.maximize_window()

# Base URL for the web scraping. Open a new window
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)

In [9]:
dataframes_list = []

url_base = "https://infogob.jne.gob.pe/Eleccion"
# Prepare iteration for all options in "Eleccion"
for number in range(2, 27):
    #First, we go to the website and display dropdown "Tipo de proceso"
    driver.get(url_base)
    proceso = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    proceso.click()
    time.sleep(1)
    # Then select "Elecciones Presidenciales"
    presidenciales = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidenciales.click()
    time.sleep(1)
    # Then display dropdown "Eleccion"
    eleccion=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()
    time.sleep(1)
    # Then select an option in "Eleccion" for the iteration
    election_1 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(number))
    eleccion_name = election_1.text
    print(f'Nombre de la elección: {eleccion_name}')  # Verificatiión
    election_1.click()
    time.sleep(1)
    # Click in "Ver datos de la eleccion"
    ver_datos = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    ver_datos.click()
    time.sleep(2)
    # Then click on tab "Candidatos y Resultados"
    resultado = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultado.click()
    time.sleep(1)
    # Extract information in table
    table_path = driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute('innerHTML')

    # Create headers to avoid issues with read_html
    head = "<table id=\"gridEleccionResultadosElectorales\" class=\"table table-hover table-striped\">"
    table_new = head + table_html[21:-1]
    table = pd.read_html(table_new)
    table_clean = table[0].iloc[:, :]
    table_clean = table_clean.loc[:, ['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    # Insert the election name into the first column
    table_clean.insert(0, 'Elecciones', eleccion_name)
    print(table_clean.head())  # Verificación adicional

    # Add the cleaned DataFrame to the list
    dataframes_list.append(table_clean)

Nombre de la elección: 


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00007FF7E0999642+30946]
	(No symbol) [0x00007FF7E094E3D9]
	(No symbol) [0x00007FF7E0846E09]
	(No symbol) [0x00007FF7E0899712]
	(No symbol) [0x00007FF7E088C271]
	(No symbol) [0x00007FF7E08BCA6A]
	(No symbol) [0x00007FF7E088BBB6]
	(No symbol) [0x00007FF7E08BCC80]
	(No symbol) [0x00007FF7E08DB041]
	(No symbol) [0x00007FF7E08BC813]
	(No symbol) [0x00007FF7E088A6E5]
	(No symbol) [0x00007FF7E088B021]
	GetHandleVerifier [0x00007FF7E0ACF84D+1301229]
	GetHandleVerifier [0x00007FF7E0ADBDC7+1351783]
	GetHandleVerifier [0x00007FF7E0AD2A13+1313971]
	GetHandleVerifier [0x00007FF7E09CDD16+245686]
	(No symbol) [0x00007FF7E095759F]
	(No symbol) [0x00007FF7E0953814]
	(No symbol) [0x00007FF7E09539A2]
	(No symbol) [0x00007FF7E094A3FF]
	BaseThreadInitThunk [0x00007FFDC005257D+29]
	RtlUserThreadStart [0x00007FFDC152AF28+40]


In [38]:
# TTransform the list of DataFrames into a single DataFrame
df_vote = pd.concat(dataframes_list, ignore_index=True)

In [40]:
# Save the result and export it to an Excel file with the group name
file_name = 'group_7_ass_5_2024_2.xlsx'
df_vote.to_excel(file_name, index=False)

# Load the Excel file
workbook = load_workbook(file_name)
worksheet = workbook.active

# Adjust the width of the first three columns
for col in ['A', 'B', 'C']:
    worksheet.column_dimensions[col].width = 30

# Save the changes to the Excel file
workbook.save(file_name)